# Med-GrapherNCA: Multi-Level Experiments
### All 6 combinations: b1b1, m1b1, m1m1, m2b1, m2m2, m1m2

This notebook trains multi-level (2-level) model combinations on the ISIC 2018 dataset.
Level 0 operates on 64x64 downsampled images, Level 1 operates on 256x256 patches.

---

## 0. Google Colab Setup: Mount Drive & Download Dataset

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Setup project directory in Google Drive
PROJECT_ROOT = '/content/drive/MyDrive/Experiments/Grapher_NCA'
DATASET_DIR = os.path.join(PROJECT_ROOT, 'datasets', 'ISIC2018')
MODELS_DIR = os.path.join(PROJECT_ROOT, 'Models')

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

print(f'Project root: {PROJECT_ROOT}')
print(f'Dataset dir:  {DATASET_DIR}')
print(f'Models dir:   {MODELS_DIR}')

In [ ]:
%%bash -s "$DATASET_DIR"
DATASET_DIR=$1

# Download ISIC 2018 Task 1 Training Data (images + masks)
if [ ! -d "$DATASET_DIR/ISIC2018_Task1-2_Training_Input" ]; then
    echo "Downloading ISIC 2018 training images..."
    wget -q --show-progress -O "$DATASET_DIR/train_input.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1-2_Training_Input.zip"
    echo "Extracting training images..."
    unzip -q "$DATASET_DIR/train_input.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/train_input.zip"
else
    echo "Training images already exist, skipping download."
fi

if [ ! -d "$DATASET_DIR/ISIC2018_Task1_Training_GroundTruth" ]; then
    echo "Downloading ISIC 2018 training ground truth..."
    wget -q --show-progress -O "$DATASET_DIR/train_gt.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1_Training_GroundTruth.zip"
    echo "Extracting training ground truth..."
    unzip -q "$DATASET_DIR/train_gt.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/train_gt.zip"
else
    echo "Training ground truth already exists, skipping download."
fi

# Download ISIC 2018 Task 1 Test Data (images + masks)
if [ ! -d "$DATASET_DIR/ISIC2018_Task1-2_Test_Input" ]; then
    echo "Downloading ISIC 2018 test images..."
    wget -q --show-progress -O "$DATASET_DIR/test_input.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1-2_Test_Input.zip"
    echo "Extracting test images..."
    unzip -q "$DATASET_DIR/test_input.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/test_input.zip"
else
    echo "Test images already exist, skipping download."
fi

if [ ! -d "$DATASET_DIR/ISIC2018_Task1_Test_GroundTruth" ]; then
    echo "Downloading ISIC 2018 test ground truth..."
    wget -q --show-progress -O "$DATASET_DIR/test_gt.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1_Test_GroundTruth.zip"
    echo "Extracting test ground truth..."
    unzip -q "$DATASET_DIR/test_gt.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/test_gt.zip"
else
    echo "Test ground truth already exists, skipping download."
fi

echo "\nDataset contents:"
ls -la "$DATASET_DIR"

In [ ]:
import os, sys

# Clone repo from GitHub into Colab's /content (fast, always fresh)
REPO_DIR = '/content/grapher-nca/M3D-NCA'
if not os.path.isdir('/content/grapher-nca'):
    os.system('git clone https://github.com/AvniMittal13/grapher-nca.git /content/grapher-nca')

if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

# Install dependencies
os.system('pip install -q torchio==0.18.82 nibabel tensorboard')

print(f'Repo dir: {REPO_DIR}')
print(f'sys.path OK: {REPO_DIR in sys.path}')

## 1. Imports

In [ ]:
import torch
from src.datasets.ISIC_Dataset import ISIC2018_Dataset
from src.models.Model_BackboneNCA import BackboneNCA
from src.models.Model_GrapherNCA_M1 import GrapherNCA_M1
from src.models.Model_GrapherNCA_M2 import GrapherNCA_M2
from src.losses.LossFunctions import DiceLoss
from src.utils.Experiment import Experiment
from src.agents.Agent_Med_NCA import Agent_Med_NCA

## 2. Configure Experiment

Choose a combination to train:
- `'b1b1'` — BackboneNCA + BackboneNCA
- `'m1b1'` — Pixel-Grapher + BackboneNCA  
- `'m1m1'` — Pixel-Grapher + Pixel-Grapher
- `'m2b1'` — Patch-Grapher + BackboneNCA
- `'m2m2'` — Patch-Grapher + Patch-Grapher
- `'m1m2'` — Pixel-Grapher + Patch-Grapher

In [ ]:
# Choose multi-level combination
combination = 'm1b1'

config = [{
    'img_path': os.path.join(DATASET_DIR, 'ISIC2018_Task1-2_Training_Input'),
    'label_path': os.path.join(DATASET_DIR, 'ISIC2018_Task1_Training_GroundTruth'),
    'model_path': os.path.join(MODELS_DIR, f'GrapherNCA_multi_{combination}'),
    'device': 'cuda:0',
    'unlock_CPU': True,
    # Optimizer
    'lr': 1e-4,
    'lr_gamma': 0.9999,
    'betas': (0.5, 0.5),
    # Training
    'save_interval': 10,
    'evaluate_interval': 10,
    'n_epoch': 1000,
    'batch_size': 8,
    # Model
    'channel_n': 64,
    'inference_steps': 10,
    'cell_fire_rate': 0.5,
    'input_channels': 3,
    'output_channels': 1,
    'hidden_size': 512,
    'train_model': 1,  # Multi-level: train both levels
    # Data
    'input_size': [(64, 64), (256, 256)],
    'data_split': [0.7, 0, 0.3],
}]

## 3. Model Factory

In [ ]:
def create_model(model_code, config, device):
    """Create a model instance from a code string."""
    if model_code == 'b1':
        return BackboneNCA(
            config['channel_n'], config['cell_fire_rate'], device,
            hidden_size=config['hidden_size'], input_channels=config['input_channels']
        ).to(device)
    elif model_code == 'm1':
        return GrapherNCA_M1(
            config['channel_n'], config['cell_fire_rate'], device,
            hidden_size=config['hidden_size'], input_channels=config['input_channels'],
            k=9
        ).to(device)
    elif model_code == 'm2':
        return GrapherNCA_M2(
            config['channel_n'], config['cell_fire_rate'], device,
            hidden_size=config['hidden_size'], input_channels=config['input_channels'],
            k=9, patch_size=4
        ).to(device)
    else:
        raise ValueError(f'Unknown model code: {model_code}')

## 4. Setup and Train

In [ ]:
dataset = ISIC2018_Dataset(input_channels=config[0]['input_channels'])
device = torch.device(config[0]['device'])

# Parse combination: first 2 chars = level 0, last 2 chars = level 1
level0_code = combination[:2]
level1_code = combination[2:]

ca1 = create_model(level0_code, config[0], device)  # Downsampled level
ca2 = create_model(level1_code, config[0], device)  # Full resolution level
ca = [ca1, ca2]

agent = Agent_Med_NCA(ca)
exp = Experiment(config, dataset, ca, agent)
dataset.set_experiment(exp)
exp.set_model_state('train')
data_loader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=exp.get_from_config('batch_size'))

loss_function = DiceLoss()

# Print model info
for i, m in enumerate(ca):
    params = sum(p.numel() for p in m.parameters())
    print(f'Level {i} ({[level0_code, level1_code][i]}): {params} parameters')

In [ ]:
agent.train(data_loader, loss_function)

## 5. Evaluate

In [ ]:
agent.getAverageDiceScore(pseudo_ensemble=True, showResults=True)